In [ ]:
from data_loader import *
from evaluate_captions import *
import csv
from build_vocab import *

In [ ]:
train_json = './data/annotations/captions_train2014.json'
test_json = './data/annotations/captions_val2014.json'
train_root = './data/images/train/'
test_root = './data/images/test/'
vocab = build_vocab(train_json)
with open('TrainImageIds.csv', 'r') as f:
    reader = csv.reader(f)
    trainIds = list(reader)
trainIds = [int(i) for i in trainIds[0]]
#train_dataset = CocoDataset(train_root, train_json, trainIds, vocab)

valIds = trainIds[-len(trainIds)//5:]
trainIds = trainIds[:-len(trainIds)//5]

with open('TestImageIds.csv', 'r') as f:
    reader = csv.reader(f)
    testIds = list(reader)
testIds = [int(i) for i in testIds[0]]
#test_dataset = CocoDataset(test_root, test_json, testIds, vocab)

In [ ]:
train_loader = get_loader(train_root, train_json, trainIds, vocab, 
                          transform=None, 
                          batch_size=2, 
                          shuffle=True, 
                          num_workers=4)
val_loader = get_loader(train_root, train_json, valIds, vocab, 
                          transform=None, 
                          batch_size=2, 
                          shuffle=True, 
                          num_workers=4)
test_loader = get_loader(test_root, test_json, testIds, vocab, 
                          transform=None, 
                          batch_size=2, 
                          shuffle=True, 
                          num_workers=4)